In [3]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
#import imageio
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math

import torchvision
import torchvision.transforms as transforms
import optuna  #载入optuna优化包

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)


##optuna######
###耗费时间的过程。。。。。。。。。。。。。。。
# Define a basic convolutional layer
# 定义 ExLU 激活函数，sigma 是可学习参数
class ExLU(nn.Module):
    def __init__(self, seqFlag, trial):
        super(ExLU, self).__init__()
        # 通过 Optuna 选择 sigma 参数，范围为 2.0 到 6.0
        self.sigma = nn.Parameter(torch.tensor(trial.suggest_float(f'sigma_{seqFlag}', 2.0, 6.0), dtype=torch.float32))

    def forward(self, x):
        return torch.where(x >= 0, x, x * torch.exp(x / self.sigma))


def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)


# 定义ResNet模型
class ResNet(nn.Module):
    def __init__(self, trial, num_classes=10):
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.custom1 = ExLU(1, trial)  # 自定义激活函数层

        # Layer 1
        self.layer1_conv1 = conv3x3(16, 16)
        self.layer1_bn1 = nn.BatchNorm2d(16)
        self.layer1_conv2 = conv3x3(16, 16)
        self.layer1_bn2 = nn.BatchNorm2d(16)
        self.layer1_extra_conv1 = conv3x3(16, 16)
        self.layer1_extra_bn1 = nn.BatchNorm2d(16)
        self.layer1_extra_conv2 = conv3x3(16, 16)
        self.layer1_extra_bn2 = nn.BatchNorm2d(16)
        self.gelub11 = ExLU(2, trial)
        self.gelub12 = ExLU(3, trial)
        self.gelub13 = ExLU(4, trial)
        self.gelub14 = ExLU(5, trial)

        # Layer 2
        self.layer2_conv1 = conv3x3(16, 32, stride=2)
        self.layer2_bn1 = nn.BatchNorm2d(32)
        self.layer2_conv2 = conv3x3(32, 32)
        self.layer2_bn2 = nn.BatchNorm2d(32)
        self.layer2_extra_conv1 = conv3x3(16, 32, stride=2)  # 调整residual的通道数
        self.layer2_extra_bn1 = nn.BatchNorm2d(32)
        self.layer2_extra_conv2 = conv3x3(32, 32)
        self.layer2_extra_bn2 = nn.BatchNorm2d(32)
        self.gelub21 = ExLU(6, trial)
        self.gelub22 = ExLU(7, trial)
        self.gelub23 = ExLU(8, trial)
        self.gelub24 = ExLU(9, trial)
        self.layer2_downsample = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(32)
        )

        # Layer 3
        self.layer3_conv1 = conv3x3(32, 64, stride=2)
        self.layer3_bn1 = nn.BatchNorm2d(64)
        self.layer3_conv2 = conv3x3(64, 64)
        self.layer3_bn2 = nn.BatchNorm2d(64)
        self.layer3_extra_conv1 = conv3x3(32, 64, stride=2)  # 调整residual的通道数
        self.layer3_extra_bn1 = nn.BatchNorm2d(64)
        self.layer3_extra_conv2 = conv3x3(64, 64)
        self.layer3_extra_bn2 = nn.BatchNorm2d(64)
        self.gelub31 = ExLU(10, trial)
        self.gelub32 = ExLU(11, trial)
        self.gelub33 = ExLU(12, trial)
        self.gelub34 = ExLU(13, trial)

        self.layer3_downsample = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(64)
        )

        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # Layer 0
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.custom1(out)

        # Layer 1
        residual = out
        out = self.layer1_conv1(out)
        out = self.layer1_bn1(out)
        out = self.gelub11(out)
        out = self.layer1_conv2(out)
        out = self.layer1_bn2(out)
        out += residual
        out = self.gelub12(out)

        residual = out
        out = self.layer1_extra_conv1(out)
        out = self.layer1_extra_bn1(out)
        out = self.gelub13(out)
        out = self.layer1_extra_conv2(out)
        out = self.layer1_extra_bn2(out)
        out += residual
        out = self.gelub14(out)

        # Layer 2
        residual = out
        out = self.layer2_conv1(out)
        out = self.layer2_bn1(out)
        out = self.gelub21(out)
        out = self.layer2_conv2(out)
        out = self.layer2_bn2(out)
        out = self.gelub22(out)
        residual = self.layer2_downsample(residual)
        out += residual
        out = self.gelub22(out)

        residual = out
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out = self.gelub23(out)
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out += residual
        out = self.gelub24(out)

        # Layer 3
        residual = out
        out = self.layer3_conv1(out)
        out = self.layer3_bn1(out)
        out = self.gelub31(out)
        out = self.layer3_conv2(out)
        out = self.layer3_bn2(out)
        residual = self.layer3_downsample(residual)
        out += residual
        out = self.gelub32(out)

        residual = out
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out = self.gelub33(out)
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out += residual
        out = self.gelub34(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


learning_rate = 0.001
num_epochs = 100


# 定义目标函数
def objective(trial):
    # 模型实例化
    model = ResNet(trial)
    model.to(device)
    curr_lr = learning_rate
    # 损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器

    #history = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

    for epoch in range(num_epochs):
        optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器
        train_loss, train_correct = 0.0, 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            train_correct += (predictions == labels).sum().item()

        # Decay learning rate
        if (epoch + 1) % 20 == 0:
            curr_lr /= 3
            update_lr(optimizer, curr_lr)

        valid_loss, valid_correct = 0.0, 0
        model.eval()
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            valid_correct += (predictions == labels).sum().item()
        ##不好的trial剪枝    
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        valid_loss = valid_loss / len(test_loader.sampler)
        valid_acc = valid_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} \t Train Loss:{:.4f} Valid Loss:{:.4f} \t Train Acc:{:.2f} %  Valid Acc:{:.2f} %".format(
            epoch + 1, num_epochs,
            train_loss,
            valid_loss,
            train_acc,
            valid_acc))
        # history['train_loss'].append(train_loss)
        # history['valid_loss'].append(valid_loss)
        # history['train_acc'].append(train_acc)
        # history['valid_acc'].append(valid_acc)

    return valid_acc


study = optuna.create_study(direction='maximize')

# 运行优化过程
study.optimize(objective, n_trials=20)

# 打印最佳参数和目标值
print('Best trial:')
print('  Value: ', study.best_trial.value)
print('  Params: ')
for key, value in study.best_trial.params.items():
    print('    {}: {}'.format(key, value))

Files already downloaded and verified


[I 2024-09-28 12:44:40,995] A new study created in memory with name: no-name-ae612396-5282-4c48-b5a4-9e8ab03ff34c


Epoch:1/100 	 Train Loss:1.6233 Valid Loss:1.5446 	 Train Acc:39.21 %  Valid Acc:44.24 %
Epoch:2/100 	 Train Loss:1.3166 Valid Loss:1.1979 	 Train Acc:52.14 %  Valid Acc:56.19 %
Epoch:3/100 	 Train Loss:1.0967 Valid Loss:1.0336 	 Train Acc:60.48 %  Valid Acc:63.10 %
Epoch:4/100 	 Train Loss:0.9702 Valid Loss:0.9455 	 Train Acc:65.30 %  Valid Acc:67.17 %
Epoch:5/100 	 Train Loss:0.8643 Valid Loss:0.8286 	 Train Acc:69.36 %  Valid Acc:70.67 %
Epoch:6/100 	 Train Loss:0.7836 Valid Loss:0.8301 	 Train Acc:72.43 %  Valid Acc:72.14 %
Epoch:7/100 	 Train Loss:0.7234 Valid Loss:0.7238 	 Train Acc:74.60 %  Valid Acc:75.41 %
Epoch:8/100 	 Train Loss:0.6686 Valid Loss:0.6882 	 Train Acc:76.65 %  Valid Acc:77.03 %
Epoch:9/100 	 Train Loss:0.6258 Valid Loss:0.6380 	 Train Acc:78.04 %  Valid Acc:78.82 %
Epoch:10/100 	 Train Loss:0.5880 Valid Loss:0.6220 	 Train Acc:79.57 %  Valid Acc:78.87 %
Epoch:11/100 	 Train Loss:0.5654 Valid Loss:0.5881 	 Train Acc:80.25 %  Valid Acc:80.33 %
Epoch:12/100 	 Trai

[I 2024-09-28 14:00:18,962] Trial 0 finished with value: 87.46000000000001 and parameters: {'sigma_1': 4.117417817946505, 'sigma_2': 5.449013271348036, 'sigma_3': 2.779244364432301, 'sigma_4': 4.530349575211693, 'sigma_5': 5.967907012091073, 'sigma_6': 4.889869277518576, 'sigma_7': 5.773191060313531, 'sigma_8': 3.1520967075023574, 'sigma_9': 5.906979297861794, 'sigma_10': 2.588424490048343, 'sigma_11': 5.008784183170302, 'sigma_12': 4.927360753952874, 'sigma_13': 4.507570565685883}. Best is trial 0 with value: 87.46000000000001.


Epoch:100/100 	 Train Loss:0.1398 Valid Loss:0.4682 	 Train Acc:95.07 %  Valid Acc:87.46 %
Epoch:1/100 	 Train Loss:1.5555 Valid Loss:1.6297 	 Train Acc:42.54 %  Valid Acc:43.64 %
Epoch:2/100 	 Train Loss:1.2029 Valid Loss:1.1941 	 Train Acc:56.68 %  Valid Acc:58.49 %
Epoch:3/100 	 Train Loss:1.0201 Valid Loss:0.9900 	 Train Acc:63.65 %  Valid Acc:65.04 %
Epoch:4/100 	 Train Loss:0.9026 Valid Loss:0.8388 	 Train Acc:67.97 %  Valid Acc:70.24 %
Epoch:5/100 	 Train Loss:0.8066 Valid Loss:0.7790 	 Train Acc:71.49 %  Valid Acc:72.79 %
Epoch:6/100 	 Train Loss:0.7353 Valid Loss:0.7249 	 Train Acc:74.26 %  Valid Acc:74.81 %
Epoch:7/100 	 Train Loss:0.6851 Valid Loss:0.6999 	 Train Acc:75.83 %  Valid Acc:75.68 %
Epoch:8/100 	 Train Loss:0.6469 Valid Loss:0.6991 	 Train Acc:77.45 %  Valid Acc:76.06 %
Epoch:9/100 	 Train Loss:0.6116 Valid Loss:0.6528 	 Train Acc:78.58 %  Valid Acc:77.67 %
Epoch:10/100 	 Train Loss:0.5859 Valid Loss:0.6265 	 Train Acc:79.60 %  Valid Acc:78.64 %
Epoch:11/100 	 Tra

...

Epoch:100/100 	 Train Loss:0.1489 Valid Loss:0.4799 	 Train Acc:94.73 %  Valid Acc:86.66 %
Epoch:1/100 	 Train Loss:1.5288 Valid Loss:1.4712 	 Train Acc:43.68 %  Valid Acc:47.64 %
Epoch:2/100 	 Train Loss:1.1839 Valid Loss:1.0593 	 Train Acc:57.36 %  Valid Acc:62.05 %
Epoch:3/100 	 Train Loss:1.0268 Valid Loss:0.9675 	 Train Acc:63.12 %  Valid Acc:65.32 %
Epoch:4/100 	 Train Loss:0.9190 Valid Loss:0.8840 	 Train Acc:67.06 %  Valid Acc:69.04 %
Epoch:5/100 	 Train Loss:0.8304 Valid Loss:0.8415 	 Train Acc:70.49 %  Valid Acc:70.75 %
Epoch:6/100 	 Train Loss:0.7635 Valid Loss:0.7229 	 Train Acc:73.17 %  Valid Acc:74.86 %
Epoch:7/100 	 Train Loss:0.7022 Valid Loss:0.6706 	 Train Acc:75.19 %  Valid Acc:77.17 %
Epoch:8/100 	 Train Loss:0.6529 Valid Loss:0.6472 	 Train Acc:76.96 %  Valid Acc:78.01 %
Epoch:9/100 	 Train Loss:0.6229 Valid Loss:0.6707 	 Train Acc:78.12 %  Valid Acc:76.90 %
Epoch:10/100 	 Train Loss:0.5843 Valid Loss:0.6559 	 Train Acc:79.51 %  Valid Acc:77.79 %
Epoch:11/100 	 Tra

[I 2024-09-29 12:28:16,248] Trial 18 finished with value: 87.35000000000001 and parameters: {'sigma_1': 5.470405003288581, 'sigma_2': 4.376988088021423, 'sigma_3': 3.4406343431737767, 'sigma_4': 2.365674717692928, 'sigma_5': 2.5254749387211586, 'sigma_6': 2.591059747502902, 'sigma_7': 3.5647256656621313, 'sigma_8': 4.797334216205891, 'sigma_9': 2.577081901694997, 'sigma_10': 3.5541924011226835, 'sigma_11': 3.364818206427023, 'sigma_12': 2.6109904675759537, 'sigma_13': 2.747362372834456}. Best is trial 5 with value: 87.79.


Epoch:100/100 	 Train Loss:0.1383 Valid Loss:0.4968 	 Train Acc:95.09 %  Valid Acc:87.35 %
Epoch:1/100 	 Train Loss:1.5857 Valid Loss:1.4090 	 Train Acc:40.77 %  Valid Acc:48.31 %
Epoch:2/100 	 Train Loss:1.2133 Valid Loss:1.0512 	 Train Acc:56.01 %  Valid Acc:62.28 %
Epoch:3/100 	 Train Loss:1.0421 Valid Loss:1.0218 	 Train Acc:62.75 %  Valid Acc:63.28 %
Epoch:4/100 	 Train Loss:0.9319 Valid Loss:0.8893 	 Train Acc:67.07 %  Valid Acc:68.27 %
Epoch:5/100 	 Train Loss:0.8432 Valid Loss:0.8057 	 Train Acc:70.14 %  Valid Acc:72.63 %
Epoch:6/100 	 Train Loss:0.7655 Valid Loss:0.7680 	 Train Acc:73.04 %  Valid Acc:73.32 %
Epoch:7/100 	 Train Loss:0.7062 Valid Loss:0.7024 	 Train Acc:75.35 %  Valid Acc:75.91 %
Epoch:8/100 	 Train Loss:0.6582 Valid Loss:0.6620 	 Train Acc:77.14 %  Valid Acc:78.00 %
Epoch:9/100 	 Train Loss:0.6159 Valid Loss:0.6097 	 Train Acc:78.49 %  Valid Acc:79.52 %
Epoch:10/100 	 Train Loss:0.5866 Valid Loss:0.6277 	 Train Acc:79.62 %  Valid Acc:78.91 %
Epoch:11/100 	 Tra

[I 2024-09-29 13:41:15,224] Trial 19 finished with value: 87.46000000000001 and parameters: {'sigma_1': 4.278433278421295, 'sigma_2': 3.371086291845102, 'sigma_3': 4.1453050307834065, 'sigma_4': 3.7808569514257533, 'sigma_5': 3.4731121517127894, 'sigma_6': 3.442913761000586, 'sigma_7': 4.081345804209523, 'sigma_8': 5.533530113840385, 'sigma_9': 3.3353561409624635, 'sigma_10': 4.215631028027772, 'sigma_11': 2.482728392351857, 'sigma_12': 3.659540972160993, 'sigma_13': 4.013921412857482}. Best is trial 5 with value: 87.79.


Epoch:100/100 	 Train Loss:0.1399 Valid Loss:0.4941 	 Train Acc:95.01 %  Valid Acc:87.46 %
Best trial:
  Value:  87.79
  Params: 
    sigma_1: 3.9858475313312236
    sigma_2: 5.2531078598099485
    sigma_3: 5.089494814479712
    sigma_4: 2.8074438391635086
    sigma_5: 2.2022404256941384
    sigma_6: 2.8515413976412893
    sigma_7: 4.4484960003692535
    sigma_8: 5.070710170952079
    sigma_9: 2.894160101529802
    sigma_10: 4.369134671365424
    sigma_11: 2.698460011357618
    sigma_12: 2.9889064668682894
    sigma_13: 2.7814107653406643


In [1]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math
import torchvision
import torchvision.transforms as transforms
import optuna  # 载入 optuna 优化包

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 图像预处理模块
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

# CIFAR-10 数据集
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor())

# 数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)

# 自定义 Raylu 激活函数类
class Raylu(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigma = nn.Parameter(2 * torch.randn(1))  # 可学习参数 sigma

    def forward(self, input):
        x = torch.where(input >= 0, input, input * torch.exp(-input ** 2 / self.sigma ** 2 / 2))
        return x

# 3x3 卷积层
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)

# 残差块
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.Raylu1 = Raylu()
        self.Raylu2 = Raylu()
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.Raylu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.Raylu2(out)
        return out

# ResNet 模型
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.Raylu3 = Raylu()
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.Raylu3(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# 模型实例化
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 初始化 Raylu 激活函数中的 sigma 参数
z = model.Raylu3.sigma
nn.init.constant_(z, 6)
z101 = model.layer1[0].Raylu1.sigma
nn.init.constant_(z101, 4)
z102 = model.layer1[0].Raylu2.sigma
nn.init.constant_(z102, 4)
z111 = model.layer1[1].Raylu1.sigma
nn.init.constant_(z111, 3)
z112 = model.layer1[1].Raylu2.sigma
nn.init.constant_(z112, 3)
z201 = model.layer2[0].Raylu1.sigma
nn.init.constant_(z201, 5)
z202 = model.layer2[0].Raylu2.sigma
nn.init.constant_(z202, 5)
z211 = model.layer2[1].Raylu1.sigma
nn.init.constant_(z211, 5)
z212 = model.layer2[1].Raylu2.sigma
nn.init.constant_(z212, 5)
z301 = model.layer3[0].Raylu1.sigma
nn.init.constant_(z301, 2)
z302 = model.layer3[0].Raylu2.sigma
nn.init.constant_(z302, 4)
z311 = model.layer3[1].Raylu1.sigma
nn.init.constant_(z311, 2)
z312 = model.layer3[1].Raylu2.sigma
nn.init.constant_(z312, 3)

# 更新学习率函数
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# 训练模型
total_step = len(train_loader)
curr_lr = learning_rate

history = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}
num_epochs = 80

for epoch in range(num_epochs):
    train_loss, train_correct = 0.0, 0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(outputs.data, 1)
        train_correct += (predictions == labels).sum().item()

    # 每 20 轮衰减学习率
    if (epoch + 1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

    valid_loss, val_correct = 0.0, 0
    model.eval()
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        valid_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(outputs.data, 1)
        val_correct += (predictions == labels).sum().item()

    train_loss /= len(train_loader.sampler)
    train_acc = train_correct / len(train_loader.sampler) * 100
    valid_loss /= len(test_loader.sampler)
    valid_acc = val_correct / len(test_loader.sampler) * 100

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, "
          f"Train Acc: {train_acc:.2f}%, Valid Acc: {valid_acc:.2f}%")

    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)
    history['train_acc'].append(train_acc)
    history['valid_acc'].append(valid_acc)


Files already downloaded and verified
Epoch [1/80], Train Loss: 1.6510, Valid Loss: 1.4344, Train Acc: 38.62%, Valid Acc: 47.45%
Epoch [2/80], Train Loss: 1.2280, Valid Loss: 1.0794, Train Acc: 55.61%, Valid Acc: 61.53%
Epoch [3/80], Train Loss: 0.9942, Valid Loss: 0.9287, Train Acc: 64.46%, Valid Acc: 67.08%
Epoch [4/80], Train Loss: 0.8649, Valid Loss: 0.8015, Train Acc: 69.40%, Valid Acc: 71.98%
Epoch [5/80], Train Loss: 0.7708, Valid Loss: 0.7369, Train Acc: 72.96%, Valid Acc: 74.62%
Epoch [6/80], Train Loss: 0.6855, Valid Loss: 0.7360, Train Acc: 76.24%, Valid Acc: 75.53%
Epoch [7/80], Train Loss: 0.6239, Valid Loss: 0.6345, Train Acc: 78.12%, Valid Acc: 78.65%
Epoch [8/80], Train Loss: 0.5756, Valid Loss: 0.6337, Train Acc: 79.75%, Valid Acc: 78.94%
Epoch [9/80], Train Loss: 0.5403, Valid Loss: 0.5981, Train Acc: 81.07%, Valid Acc: 79.79%
Epoch [10/80], Train Loss: 0.5121, Valid Loss: 0.5507, Train Acc: 82.20%, Valid Acc: 81.54%
Epoch [11/80], Train Loss: 0.4869, Valid Loss: 0.58